In [1]:
!pip install datasets

!pip install transformers

!pip install accelerate -U
!pip install transformers[torch]
!pip install torch -U
!pip install trl

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 521.2/521.2 kB 5.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 115.3/115.3 kB 13.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 10.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 261.4/261.4 kB 4.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 670.2/670.2 MB 2.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 23.7/23.7 MB 48.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 823.6/823.6 kB 54.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 14.1/14.1 MB 68.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 731.7/731.7 MB 2.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 410.6/410.6 MB 4.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 121.6/121.6 MB 8.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.5/56.5 MB 12.0 MB/s eta 0:00:00
     ━━━━━━━━━━━

In [1]:
from datasets import load_dataset

dataset = load_dataset("davidgaofc/MedQuad_split")
dataset = dataset.rename_column("Question", "query")
dataset = dataset.remove_columns(["qtype", "Answer"])


Extracting data files:   0%|          | 0/6 [00:00<?, ?it/s]

Generating SFT_train1 split:   0%|          | 0/5742 [00:00<?, ? examples/s]

Generating SFT_train2 split:   0%|          | 0/5742 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/1640 [00:00<?, ? examples/s]

Generating RL split:   0%|          | 0/1640 [00:00<?, ? examples/s]

Generating RM_oos split:   0%|          | 0/820 [00:00<?, ? examples/s]

Generating Shadow_oos split:   0%|          | 0/820 [00:00<?, ? examples/s]

In [2]:
from trl import AutoModelForSeq2SeqLMWithValueHead, PPOConfig, PPOTrainer
config = PPOConfig(
    model_name ="davidgaofc/SFT_Med_t",
    learning_rate = 2e-5,
    batch_size = 8,
)

/usr/local/lib/python3.10/dist-packages/trl/trainer/ppo_config.py:141: UserWarning: The `optimize_cuda_cache` arguement will be deprecated soon, please use `optimize_device_cache` instead.
  warnings.warn(


In [3]:
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM

tokenizer = AutoTokenizer.from_pretrained(config.model_name)

model = AutoModelForSeq2SeqLMWithValueHead.from_pretrained(config.model_name)

tokenizer.pad_token = tokenizer.eos_token

tokenizer_config.json:   0%|          | 0.00/20.7k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/2.42M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/2.20k [00:00<?, ?B/s]

config.json:   0%|          | 0.00/1.52k [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/242M [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/133 [00:00<?, ?B/s]

In [4]:
from transformers import pipeline

reward_model = pipeline("text-classification", model = "davidgaofc/RM_base")


config.json:   0%|          | 0.00/769 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/328M [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/1.22k [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/798k [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/2.11M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/280 [00:00<?, ?B/s]

In [5]:
def tokenize(sample):
    sample["input_ids"] = tokenizer(sample["query"])
    return sample
# dataset = dataset.to_dataset()
dataset = dataset.map(tokenize, batched=False)

Map:   0%|          | 0/5742 [00:00<?, ? examples/s]

Map:   0%|          | 0/5742 [00:00<?, ? examples/s]

Map:   0%|          | 0/1640 [00:00<?, ? examples/s]

Map:   0%|          | 0/1640 [00:00<?, ? examples/s]

Map:   0%|          | 0/820 [00:00<?, ? examples/s]

Map:   0%|          | 0/820 [00:00<?, ? examples/s]

In [6]:
def collator(data):
    return dict((key, [d[key] for d in data]) for key in data[0])

In [7]:
ppo_trainer = PPOTrainer(
    model=model,
    config=config,
    dataset=dataset["RL"],
    tokenizer=tokenizer,
    data_collator=collator,
)

In [8]:
generation_kwargs = {
    "min_length": -1,
    "top_k": 0.0,
    "top_p": 1.0,
    "do_sample": True,
    "pad_token_id": tokenizer.eos_token_id,
}

In [17]:
from tqdm import tqdm
import torch

for epoch, batch in tqdm(enumerate(ppo_trainer.dataloader)):
    query_tensors = batch["input_ids"]
    query_tensors = [torch.tensor(element["input_ids"]) for element in query_tensors]

    #### Get response from SFTModel
    # print(query_tensors[0]['input_ids'])
    response_tensors = ppo_trainer.generate(query_tensors, **generation_kwargs)
    batch["response"] = [tokenizer.decode(r.squeeze()) for r in response_tensors]

    #### Compute reward score
    texts = [q + r for q, r in zip(batch["query"], batch["response"])]
    pipe_outputs = reward_model(texts)
    # print(pipe_outputs)
    rewards = [torch.tensor(output["score"]) for output in pipe_outputs]

    #### Run PPO step
    stats = ppo_trainer.step(query_tensors, response_tensors, rewards)
    ppo_trainer.log_stats(stats, batch, rewards)


35it [05:05,  7.69s/it]/usr/local/lib/python3.10/dist-packages/trl/trainer/ppo_trainer.py:1200: UserWarning: The average ratio of batch (6739.52) exceeds threshold 10.00. Skipping batch.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/trl/trainer/ppo_trainer.py:1200: UserWarning: The average ratio of batch (6671.40) exceeds threshold 10.00. Skipping batch.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/trl/trainer/ppo_trainer.py:1200: UserWarning: The average ratio of batch (6662.93) exceeds threshold 10.00. Skipping batch.
  warnings.warn(
37it [05:22,  8.00s/it]/usr/local/lib/python3.10/dist-packages/trl/trainer/ppo_trainer.py:1200: UserWarning: The average ratio of batch (15.65) exceeds threshold 10.00. Skipping batch.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/trl/trainer/ppo_trainer.py:1200: UserWarning: The average ratio of batch (18.82) exceeds threshold 10.00. Skipping batch.
  warnings.warn(
45it [06:29,  8.63s/it]/usr/local/lib/python3.10/dist

AttributeError: ignored

In [18]:
ppo_trainer.model.save_pretrained("new")

In [19]:
from huggingface_hub import notebook_login

notebook_login()

In [22]:

ppo_trainer.push_to_hub("davidgaofc/PPO_base")

model.safetensors:   0%|          | 0.00/242M [00:00<?, ?B/s]

'https://huggingface.co/davidgaofc/PPO_base/tree/main/'